In [1]:
import pandas as pd
import seaborn as sns

train = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/train.csv")
test = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/test.csv")

log_train = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/log_train.csv")
log_test = pd.read_csv("/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/log_test.csv")

In [2]:
log_train_df= log_train
train_df = train

In [3]:
log_train_df['time'] = pd.to_datetime(log_train_df['time'])

# Extract features
log_train_df['hour'] = log_train_df['time'].dt.hour
log_train_df['day_of_week'] = log_train_df['time'].dt.dayofweek
log_train_df['day'] = log_train_df['time'].dt.day

# Count of events per ID
event_count = log_train_df.groupby('ID')['event'].count().reset_index(name='event_count')

# Count of unique sources per ID
unique_sources = log_train_df.groupby('ID')['source'].nunique().reset_index(name='unique_sources')

# Merge these features back to the original dataset on ID
train_df_enriched = train_df.merge(event_count, on='ID', how='left')
train_df_enriched = train_df_enriched.merge(unique_sources, on='ID', how='left')


# Time difference features (e.g., average time between events for each ID)
log_train_df['time_diff'] = log_train_df.sort_values(['ID', 'time']).groupby('ID')['time'].diff().dt.total_seconds()
avg_time_diff = log_train_df.groupby('ID')['time_diff'].mean().reset_index(name='avg_time_diff')

# Fill missing values for IDs that did not have event data


train_df_enriched = train_df_enriched.merge(avg_time_diff, on='ID', how='left')

# Fill missing values for IDs that did not have event data
train_df_enriched.fillna({'event_count': 0, 'unique_sources': 0, "avg_time_diff":0}, inplace=True)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


In [4]:
train_df_enriched.head()

,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X134,X135,X136,X137,X138,X139,label,event_count,unique_sources,avg_time_diff
0,7054.0,2.015152,0.0,32.0,NaN,4.0,NaN,NaN,7.0,6.0,...,1.0,1.0,1.0,3.0,2.0,NaN,1,0.0,0.0,0.000000
1,147131.0,1.000000,NaN,2.0,NaN,NaN,0.0,NaN,6.0,5.0,...,3.0,1.0,1.0,2.0,10.0,NaN,1,1.0,1.0,0.000000
2,132068.0,1.000000,NaN,1.0,NaN,2.0,NaN,0.0,9.0,1.0,...,1.0,1.0,2.0,2.0,4.0,NaN,1,0.0,0.0,0.000000
3,85195.0,2.071429,0.0,1.0,NaN,2.0,NaN,NaN,8.0,7.0,...,2.0,2.0,1.0,4.0,2.0,NaN,1,40.0,2.0,1182.717949
4,191948.0,2.000000,NaN,2.0,0.0,2.0,NaN,NaN,14.0,2.0,...,2.0,2.0,1.0,2.0,6.0,0.0,1,0.0,0.0,0.000000


In [31]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import ADASYN

# Assuming 'train_df_enriched' is your dataset
train_clean = train_df_enriched.dropna(subset=['label'])

X = train_clean.drop(['label', 'ID'], axis=1)
y = train_clean['label']

# Initialize the SimpleImputer with 'median' strategy
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)
X_imputed_df = pd.DataFrame(X_imputed, columns=X.columns)

# Resampling
X_resampled, y_resampled = ADASYN().fit_resample(X_imputed_df, y)

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = 0.2, random_state = 42)

# Define improved model parameters for 'dart' boosting
params = {
    'boosting_type': 'dart',  # Use 'dart' boosting type
    'objective': 'binary',
    'metric': 'binary_logloss',
    'learning_rate': 0.01,
    'num_leaves': 40,
    'max_depth': -1,
    'min_child_samples': 30,
    'max_bin': 255,
    'subsample': 0.8,
    'subsample_freq': 1,
    'colsample_bytree': 0.8,
    'min_child_weight': 0.001,
    'subsample_for_bin': 200000,
    'min_split_gain': 0.0,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'verbose': -1
}

# Creating the LightGBM data containers
dtrain = lgb.Dataset(X_train, label=y_train)
dtest = lgb.Dataset(X_test, label=y_test, reference=dtrain)

# Train the model
clf = lgb.train(params, dtrain, num_boost_round=2000, valid_sets=[dtrain, dtest],callbacks=[lgb.early_stopping(stopping_rounds=100),lgb.log_evaluation(period=100)])

# Predictions
predictions = clf.predict(X_test, num_iteration=clf.best_iteration)

# Evaluation
accuracy = accuracy_score(y_test, predictions.round())
print(f"Accuracy: {accuracy:.2f}")
print(confusion_matrix(y_test, predictions.round()))
print(classification_report(y_test, predictions.round()))


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	training's binary_logloss: 0.511727	valid_1's binary_logloss: 0.513063
[200]	training's binary_logloss: 0.446951	valid_1's binary_logloss: 0.448905
[300]	training's binary_logloss: 0.389546	valid_1's binary_logloss: 0.391914
[400]	training's binary_logloss: 0.346799	valid_1's binary_logloss: 0.349625
[500]	training's binary_logloss: 0.310928	valid_1's binary_logloss: 0.313986
[600]	training's binary_logloss: 0.2987	valid_1's binary_logloss: 0.301928
[700]	training's binary_logloss: 0.278809	valid_1's binary_logloss: 0.282199
[800]	training's binary_logloss: 0.267647	valid_1's binary_logloss: 0.271221
[900]	training's binary_logloss: 0.254577	valid_1's binary_logloss: 0.258423
[1000]	training's binary_logloss: 0.24718	valid_1's binary_logloss: 0.25138
[1100]	training's binary_logloss: 0.239449	valid_1's binary_logloss: 0.244134
[1200]	training's binary_logloss: 0.235297	valid_1's binary_logloss: 0.240338
[1300]	training's binary_logloss: 0.230167	valid_1's binary_logloss: 0.23572


In [32]:
from sklearn.metrics import f1_score

# Assuming `predictions` are the predicted probabilities for the positive class from your model
# And `y_test` are the true binary labels in the test set

# Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
binary_predictions = [1 if prob > 0.5 else 0 for prob in predictions]

# Calculate F1 score
f1 = f1_score(y_test, binary_predictions)

print(f'F1 Score: {f1}')
#F1_Score : 0.9221600231637873 
#F1_Score : 0.9229871645274212 - 0.1 median
#F1_Score : 0.9242930498300427 - mean lightbgm
#F1_Score : 0.9243412961382085 - median lightbgm
#F1_Score : 0.9246502317282795 - lightbgm median learning rate= 0.01
#F1_Score : 0.9207335755241388 - lightbgm median learning rate= 0.01 dart
#F1 Score: 0.9220892274211099 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100
#F1 Score: 0.924385564845984 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=1400
#F1 Score: 0.9251454801749377 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=1800 *
#F1 Score: 0.9258336042773021 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=2200
#F1 Score: 0.9257118080647493 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=2000

F1 Score: 0.9257118080647493


In [33]:
from sklearn.metrics import roc_auc_score

# Assuming `predictions` are the predicted probabilities for the positive class from your model
# And `y_test` are the true binary labels in the test set

# Calculate AUC score
auc_score = roc_auc_score(y_test, predictions)

print(f'AUC Score: {auc_score}')
#AUC Score: 0.9234141033971461 - lightbgm median learning rate= 0.01
#AUC Score: 0.9198248761840062 - lightbgm median learning rate= 0.01 dart
#AUC Score: 0.9646368563832868 - lightbgm median learning rate= 0.01 dart without fold
#AUC Score: 0.9645827863163663 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100
#AUC Score: 0.9659848237540357 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=1400
#AUC Score: 0.9666748373275778 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=1800 *
#AUC Score: 0.9670271115081299 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=2200
#AUC Score: 0.9668365984505018 - lightbgm median learning rate= 0.01 dart - with stopping_rounds=100 - num_boost_round=2000

AUC Score: 0.9668365984505018


In [11]:
log_test_df=log_test
test_df = test

In [12]:
log_test_df['time'] = pd.to_datetime(log_test_df['time'])

# Extract features
log_test_df['hour'] = log_test_df['time'].dt.hour
log_test_df['day_of_week'] = log_test_df['time'].dt.dayofweek
log_test_df['day'] = log_test_df['time'].dt.day

# Count of events per ID
event_count = log_test_df.groupby('ID')['event'].count().reset_index(name='event_count')

# Count of unique sources per ID
unique_sources = log_test_df.groupby('ID')['source'].nunique().reset_index(name='unique_sources')

# Merge these features back to the original dataset on ID
test_df_enriched = test_df.merge(event_count, on='ID', how='left')
test_df_enriched = test_df_enriched.merge(unique_sources, on='ID', how='left')



# Time difference features (e.g., average time between events for each ID)
log_test_df['time_diff'] = log_test_df.sort_values(['ID', 'time']).groupby('ID')['time'].diff().dt.total_seconds()
avg_time_diff = log_test_df.groupby('ID')['time_diff'].mean().reset_index(name='avg_time_diff')



test_df_enriched = test_df_enriched.merge(avg_time_diff, on='ID', how='left')

# Fill missing values for IDs that did not have event data
test_df_enriched.fillna({'event_count': 0, 'unique_sources': 0, 'avg_time_diff':0}, inplace=True)

In [34]:
# Assuming `test_df_enriched` is your test dataset and `clf` is your trained LightGBM model

# Convert the 'ID' column from float to int
test_df_enriched['ID'] = test_df_enriched['ID'].astype(int)

# Separate out the 'ID' column and features from the 'test' dataset
test_features = test_df_enriched.drop('ID', axis=1)

# Impute the 'test' dataset using the fitted SimpleImputer
# Note: We use .transform() here, NOT .fit_transform()
test_imputed = imputer.transform(test_features)

# Predict the labels using the trained LightGBM model
predicted_labels = clf.predict(test_imputed)

# For binary classification, LightGBM might output probabilities. If so, convert these to 0 or 1
# Uncomment the following line if your model outputs probabilities
predicted_labels = np.round(predicted_labels).astype(int)

# Create a DataFrame with 'ID' and the predicted 'label'
results_df = pd.DataFrame({
    'ID': test_df_enriched['ID'],
    'LABEL': predicted_labels
})

# Optionally, convert the 'LABEL' column to the desired format, e.g., int
# results_df['LABEL'] = results_df['LABEL'].astype(int)

# Output the DataFrame with 'ID' and 'label'
# Save to a CSV file
results_df.to_csv('/Users/mohan/NEU/Projects_and_Hackathon/stay-or-stray/Submissions/my_submission_27_lGBM_ADAS_mdn_0.01LR_dart_wo_fold_2000BR.csv', index=False)

# Display the first few rows of the DataFrame
print(results_df.head())


       ID  LABEL
0   17547      0
1  140449      1
2  182658      1
3  149652      1
4  106304      1


## Optuna:

In [38]:
import optuna
import lightgbm as lgb
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# Assuming X_resampled and y_resampled are your features and target after resampling

def objective(trial):
    # Splitting data for hyperparameter optimization
    X_train, X_valid, y_train, y_valid = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

    # Defining hyperparameter search space
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'dart',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
    }

    # Training the model with the trial's parameters
    gbm = lgb.train(param, lgb.Dataset(X_train, label=y_train),num_boost_round=2000, valid_sets=[lgb.Dataset(X_valid, label=y_valid)],callbacks=[lgb.early_stopping(stopping_rounds=100),lgb.log_evaluation(period=100)])
    preds = gbm.predict(X_valid)
    # Calculating log loss for the validation set
    loss = log_loss(y_valid, preds)
    return loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100) # You can adjust the number of trials

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


[I 2024-03-27 01:36:48,476] A new study created in memory with name: no-name-a846d352-c07a-4a05-822a-1e47d5119479
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	valid_0's binary_logloss: 0.2098
[200]	valid_0's binary_logloss: 0.204803
[300]	valid_0's binary_logloss: 0.203804
[400]	valid_0's binary_logloss: 0.204209
[500]	valid_0's binary_logloss: 0.204418
[600]	valid_0's binary_logloss: 0.204824
[700]	valid_0's binary_logloss: 0.205846
[800]	valid_0's binary_logloss: 0.206643
[900]	valid_0's binary_logloss: 0.208074
[1000]	valid_0's binary_logloss: 0.209357
[1100]	valid_0's binary_logloss: 0.210251
[1200]	valid_0's binary_logloss: 0.210779
[1300]	valid_0's binary_logloss: 0.211633
[1400]	valid_0's binary_logloss: 0.213672
[1500]	valid_0's binary_logloss: 0.21422
[1600]	valid_0's binary_logloss: 0.215369
[1700]	valid_0's binary_logloss: 0.21639
[1800]	valid_0's binary_logloss: 0.217886
[1900]	valid_0's binary_logloss: 0.219037
[2000]	valid_0's binary_logloss: 0.219662


[I 2024-03-27 01:45:10,591] Trial 0 finished with value: 0.219662168867634 and parameters: {'learning_rate': 0.1966881323278584, 'num_leaves': 48, 'max_depth': 8, 'min_child_samples': 91, 'subsample': 0.8161026202884337, 'colsample_bytree': 0.41374602742460775, 'reg_alpha': 0.3077145674943904, 'reg_lambda': 0.374859826307127}. Best is trial 0 with value: 0.219662168867634.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	valid_0's binary_logloss: 0.208047
[200]	valid_0's binary_logloss: 0.207516
[300]	valid_0's binary_logloss: 0.210665
[400]	valid_0's binary_logloss: 0.217622
[500]	valid_0's binary_logloss: 0.225385
[600]	valid_0's binary_logloss: 0.226924
[700]	valid_0's binary_logloss: 0.23247
[800]	valid_0's binary_logloss: 0.235829
[900]	valid_0's binary_logloss: 0.241559
[1000]	valid_0's binary_logloss: 0.244534
[1100]	valid_0's binary_logloss: 0.248267
[1200]	valid_0's binary_logloss: 0.248672
[1300]	valid_0's binary_logloss: 0.250541
[1400]	valid_0's binary_logloss: 0.255137
[1500]	valid_0's binary_logloss: 0.255231
[1600]	valid_0's binary_logloss: 0.258043
[1700]	valid_0's binary_logloss: 0.260766
[1800]	valid_0's binary_logloss: 0.262336
[1900]	valid_0's binary_logloss: 0.263813
[2000]	valid_0's binary_logloss: 0.264462


[I 2024-03-27 02:04:26,122] Trial 1 finished with value: 0.26446195703149344 and parameters: {'learning_rate': 0.2519070240449133, 'num_leaves': 287, 'max_depth': 10, 'min_child_samples': 72, 'subsample': 0.7502382929293463, 'colsample_bytree': 0.611900240286269, 'reg_alpha': 0.9138258452559027, 'reg_lambda': 0.5615376369483004}. Best is trial 0 with value: 0.219662168867634.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


...

[100]	valid_0's binary_logloss: 0.220531
[200]	valid_0's binary_logloss: 0.212123
[300]	valid_0's binary_logloss: 0.20697
[400]	valid_0's binary_logloss: 0.205517
[500]	valid_0's binary_logloss: 0.20402
[600]	valid_0's binary_logloss: 0.204038
[700]	valid_0's binary_logloss: 0.203534
[800]	valid_0's binary_logloss: 0.204073
[900]	valid_0's binary_logloss: 0.204102
[1000]	valid_0's binary_logloss: 0.204453
[1100]	valid_0's binary_logloss: 0.204631
[1200]	valid_0's binary_logloss: 0.204616
[1300]	valid_0's binary_logloss: 0.204467
[1400]	valid_0's binary_logloss: 0.20467
[1500]	valid_0's binary_logloss: 0.205014
[1600]	valid_0's binary_logloss: 0.205495
[1700]	valid_0's binary_logloss: 0.205607
[1800]	valid_0's binary_logloss: 0.205519
[1900]	valid_0's binary_logloss: 0.20594
[2000]	valid_0's binary_logloss: 0.206193


[I 2024-03-27 04:22:50,725] Trial 12 finished with value: 0.20619290635785875 and parameters: {'learning_rate': 0.2909025512653034, 'num_leaves': 109, 'max_depth': 3, 'min_child_samples': 58, 'subsample': 0.9944636180513621, 'colsample_bytree': 0.8083400564713437, 'reg_alpha': 0.6819327007430821, 'reg_lambda': 0.7548068399421297}. Best is trial 11 with value: 0.20383553028380824.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/callback.py:325: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[100]	valid_0's binary_logloss: 0.215084
[200]	valid_0's binary_logloss: 0.208312
[300]	valid_0's binary_logloss: 0.205688
[400]	valid_0's binary_logloss: 0.204513
[500]	valid_0's binary_logloss: 0.204595
[600]	valid_0's binary_logloss: 0.205465
[700]	valid_0's binary_logloss: 0.20549
[800]	valid_0's binary_logloss: 0.205782
[900]	valid_0's binary_logloss: 0.207062
[1000]	valid_0's binary_logloss: 0.207447
[1100]	valid_0's binary_logloss: 0.208095
[1200]	valid_0's binary_logloss: 0.208698
[1300]	valid_0's binary_logloss: 0.209473
[1400]	valid_0's binary_logloss: 0.21052
[1500]	valid_0's binary_logloss: 0.211264
[1600]	valid_0's binary_logloss: 0.212844
[1700]	valid_0's binary_logloss: 0.214188


[W 2024-03-27 04:28:02,733] Trial 13 failed with parameters: {'learning_rate': 0.21599244058772785, 'num_leaves': 92, 'max_depth': 5, 'min_child_samples': 22, 'subsample': 0.900205748327243, 'colsample_bytree': 0.7919580542295129, 'reg_alpha': 0.740733215839575, 'reg_lambda': 0.7737686498524444} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/w5/7276v8b14pj5_4lrc2r9xkvr0000gn/T/ipykernel_5469/1606589854.py", line 29, in objective
    gbm = lgb.train(param, lgb.Dataset(X_train, label=y_train),num_boost_round=2000, valid_sets=[lgb.Dataset(X_valid, label=y_valid)],callbacks=[lgb.early_stopping(stopping_rounds=100),lgb.log_evaluation(period=100)])
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/engine.py", line

KeyboardInterrupt: 